In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
from random import random
#from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

<h1>Naive Bayes Classifier</h1>

Naive Bayes is one of the simplest supervised ML algorithms meanwhile very efficient and also is able to learn fast and make a quick prediction, therefore it is so useful and popular.
Naive Bayes contains two words Naive and Bayes, Bayes because it is built on Bayes Theorem, and Naive because it assumes that features are independent even if they actually are interdependent.It is simple but very powerful and works well with large datasets and sparse matrices. It works really well on text classification problems, and spam filtering.

<h2 id='bayes_theorem'> Bayes Theorem </h2>

Bayes theorem describes a probability of an event, based on prior knowledge of conditions that might be related to an event.
First, let's take the formula of conditional probability and try to derive Bayes Theorem:

$$p(A|B) = p(B\cap A)/p(B)$$
probability of event A given B, meaning what is the probability of A when event B is already taken place, wich is eqaul to quals probability of A intersection B (the probability of both A and B events are taken place) divided by probability of B.
we have the same for probability of event B given event A $$p(B|A) = p(A\cap B)/p(A)$$
the  $p(A\cap B)$ and  $p(B\cap A)$ are basicaly the same. Since they are the same, we can get two formulas and move denominator to the left of the equation,and equate them 
$$ p(B|A)p(A) = p(A\cap B) = p(B \cap A) = p(A|B)p(B) $$

So, when we want to find probability of A given B we can write our equation this way: <br>
$P(A|B) = P(B|A) * P(A) / P(B)$, and this is the equation of Bayes Theorem

* P(A|B) is the posterior probability of class (target) given predictor (attribute).
* P(B) is the prior probability of class.
* P(B|A) is the likelihood which is the probability of predictor given class.
* P(A) is the prior probability of predictor.

In [3]:
iris = pd.read_csv("../../../resources/data/golf_df.csv")
iris

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


<IPython.core.display.Javascript object>

In [4]:
c = iris["Play"].value_counts().to_numpy()

<IPython.core.display.Javascript object>

In [5]:
probability = c / c.sum()
probability

array([0.64285714, 0.35714286])

<IPython.core.display.Javascript object>

In [6]:
# k = iris.loc['Play']=='No' #loc['viper']
# .groupby('Outlook').count()
rslt_df = iris[iris["Play"] == "yes"]
rslt_df

,Outlook,Temperature,Humidity,Windy,Play
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
6,overcast,cool,normal,True,yes
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes
10,sunny,mild,normal,True,yes
11,overcast,mild,high,True,yes
12,overcast,hot,normal,False,yes


<IPython.core.display.Javascript object>

In [7]:
iris.groupby("Outlook").count()

,Temperature,Humidity,Windy,Play
Outlook,,,,
overcast,4,4,4,4
rainy,5,5,5,5
sunny,5,5,5,5


<IPython.core.display.Javascript object>

In [8]:
rslt_df.groupby("Outlook").count()
# https://www.geeksforgeeks.org/naive-bayes-classifiers/

,Temperature,Humidity,Windy,Play
Outlook,,,,
overcast,4,4,4,4
rainy,3,3,3,3
sunny,2,2,2,2


<IPython.core.display.Javascript object>

So, in the figure above, we have calculated P(xi | yj) for each xi in X and yj in y manually in the tables 1-4. For example, probability of playing golf given that the temperature is cool, i.e P(temp. = cool | play golf = Yes) = 3/9.

Also, we need to find class probabilities (P(y)) which has been calculated in the table 5. For example, P(play golf = Yes) = 9/14.

So now, we are done with our pre-computations and the classifier is ready!

Let us test it on a new set of features (let us call it today):

Now, with regards to our dataset, we can apply Bayes’ theorem in following way:

 $$P(y|X) = \frac{P(X|y) P(y)}{P(X)}$$ 
 where, $y$ is class variable and $X$ is a dependent feature vector (of size n) where:

 $X = (x_1,x_2,x_3,.....,x_n) $

X = (Rainy, Hot, High, False) <br> <br>
y = no

So basically, P(y|X) here means, the probability of “Not playing golf” given that the weather conditions are “Rainy outlook”, “Temperature is hot”, “high humidity” and “no wind”.
With assumtion of independence we can write down for every feature,bayesin theorem : 

$$ P(y|x_1,...,x_n) = \frac{ P(x_1|y)P(x_2|y)...P(x_n|y)P(y)}{P(x_1)P(x_2)...P(x_n)} $$
which can be expressed as:
$$ P(y|x_1,...,x_n) = \frac{P(y)\prod_{i=1}^{n}P(x_i|y)}{P(x_1)P(x_2)...P(x_n)}  $$
Now, as the denominator remains constant for a given input, we can remove that term:

 $$P(y|x_1,...,x_n)\propto P(y)\prod_{i=1}^{n}P(x_i|y) $$

Now, we need to create a classifier model. For this, we find the probability of given set of inputs for all possible values of the class variable y and pick up the output with maximum probability. This can be expressed mathematically as:

In [9]:
df = iris

<IPython.core.display.Javascript object>

In [10]:
df.groupby("Outlook")["Play"].count().to_numpy()

array([4, 5, 5], dtype=int64)

<IPython.core.display.Javascript object>

In [11]:
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


<IPython.core.display.Javascript object>

In [12]:
label = "Play"
yes = df[df[label] == "yes"].groupby("Outlook")[label].count()
no = df[df[label] == "no"].groupby("Outlook")[label].count()

<IPython.core.display.Javascript object>

In [223]:
likehood_table = {}
def get_value_feature(df,feature):
    try:
        return df[feature]
    except:
        return 0


<IPython.core.display.Javascript object>

In [224]:
def create_likehood_tb(df, label):
    likehood_table = {}
    features = df.drop(label, axis=1).columns
    for feature in features:
        yes = df[df[label] == "yes"].groupby(feature)[label].count()
        no = df[df[label] == "no"].groupby(feature)[label].count()
        all = df.groupby(feature)[label].count()
        for feature_value in all.index:
            c = all[feature_value]
            c1 = get_value_feature(yes, feature_value)
            c2 = get_value_feature(no, feature_value)
            likehood_table[feature_value] = {
                "yes": c1 / yes.sum(),
                "no": c2 / no.sum(),
                "probability": c / all.sum(),
            }
    return likehood_table

<IPython.core.display.Javascript object>

In [225]:
likehood_df = create_likehood_tb(df, "Play")

<IPython.core.display.Javascript object>

In [226]:
likehood_df = pd.DataFrame(likehood_df)
likehood_df

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
yes,0.444444,0.333333,0.222222,0.333333,0.222222,0.444444,0.333333,0.666667,0.666667,0.333333
no,0.000000,0.400000,0.600000,0.200000,0.400000,0.400000,0.800000,0.200000,0.400000,0.600000
probability,0.285714,0.357143,0.357143,0.285714,0.285714,0.428571,0.500000,0.500000,0.571429,0.428571


<IPython.core.display.Javascript object>

In [238]:
test = df.drop("Play", axis=1)


def predict(args):
    yes = 0.642857
    no = 0.357143
    for i in args:
        yes = yes * likehood_df[i]["yes"]
        # print(i, likehood_df[i]["yes"])
        no = no * likehood_df[i]["no"]
        # print(i, likehood_df[i]["no"])

    return "yes" if yes > no else "no"

<IPython.core.display.Javascript object>

In [239]:
result = test.apply(predict, axis=1)

<IPython.core.display.Javascript object>

In [242]:
from sklearn.metrics import accuracy_score
    


<IPython.core.display.Javascript object>

In [244]:
accuracy_score(df["Play"],result)

0.9285714285714286

<IPython.core.display.Javascript object>

We need to find $P(x_i | y_j)$ for each $xi$ in $X$ and $y_j$ in $y.$ All these calculations have been demonstrated in the tables below:

In [16]:
all = df.groupby("Outlook")["Play"].count()

<IPython.core.display.Javascript object>

<h2> References </h2>
* <a href='https://towardsdatascience.com/implementing-naive-bayes-algorithm-from-scratch-python-c6880cfc9c41'>naive <br>
* <a href='https://prwatech.in/blog/machine-learning/naive-bayes-classifier-in-machine-learning/'> Indian Naive Bayes </a> <br>
* <a href='https://www.geeksforgeeks.org/naive-bayes-classifiers/'>Naive Bayes Classifiers </a>    